In [1]:
!pip install transformers

In [2]:
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertModel, BertForSequenceClassification

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
# Test sentence
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
input_sentence = "I love my AirPods Pro"
encoded = tokenizer(input_sentence)
print('Input sentence: ', input_sentence, '\n')
print('Encoded: ', encoded, '\n')
print('Decoded: ', tokenizer.decode(encoded['input_ids']), '\n')



Input sentence:  Hello my name is Jin 

Encoded:  {'input_ids': [101, 7592, 2026, 2171, 2003, 9743, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]} 

Decoded:  [CLS] hello my name is jin [SEP] 



## Get Preprocessed Review Data

In [5]:
# Get all review files
file_path = '/content/drive/My Drive/BERT project/Reviews/*[0-9].csv'
files = glob.glob(file_path)

In [6]:
# Concat all review data from different products into one big dataframe
df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list, axis=0, ignore_index=True)
df

,comment,stars,verified,date,country,helpful,has-media
0,I could sit here and write all about the specs...,5,0,2019-6-25,1,3,0
1,A very reasonably priced laptop for basic comp...,4,0,2019-7-2,1,1,1
2,"This is the best laptop deal you can get, full...",5,1,2019-8-3,1,1,1
3,A few months after the purchase....It is still...,5,1,2019-7-12,1,1,0
4,BUYER BE AWARE: This computer has Microsoft 10...,1,1,2019-8-7,1,6,0
...,...,...,...,...,...,...,...
120384,Great overall product,5,1,2019-12-28,1,0,0
120385,wow~^^very good,5,1,2019-12-30,1,0,0
120386,The sound is amazing!,5,1,2020-1-9,1,0,0
120387,I love my AirPods Pro,5,1,2019-11-23,1,0,0


## Data Exploration

In [7]:
# Check for missing values
df.isnull().any()

comment       True
stars        False
verified     False
date         False
country      False
helpful      False
has-media    False
dtype: bool

In [8]:
# Find reviews with missing comments
missing_indices = df[df['comment'].isnull()].index.tolist()
print('Number of reviews missing comments: ', len(missing_indices))
print('Missing indices: ', missing_indices)

Number of reviews missing comments:  17
Missing indices:  [24835, 37237, 37277, 40072, 50852, 64895, 69760, 80854, 81562, 84103, 86420, 92670, 98539, 98769, 105389, 105405, 112139]


In [9]:
print('Max comment length (of all products): ', int(df.comment.str.len().max()))

Max comment length (of all products):  5127


In [46]:
# Get only the comments and star (labels) data
df = df[['comment', 'stars']]
df.dropna(inplace=True)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,comment,stars
0,I could sit here and write all about the specs...,5
1,A very reasonably priced laptop for basic comp...,4
2,"This is the best laptop deal you can get, full...",5
3,A few months after the purchase....It is still...,5
4,BUYER BE AWARE: This computer has Microsoft 10...,1
...,...,...
120384,Great overall product,5
120385,wow~^^very good,5
120386,The sound is amazing!,5
120387,I love my AirPods Pro,5


In [47]:
# print first 5 reviews
for idx, row in comment_df[:5].iterrows():
    print(row['comment'] + '\n')

I could sit here and write all about the specs on this computer, but they are already in the description, and If you are like me... you don't really understand it anyways.So I am going to tell you what I LOVE about this computer and what I use it for. I am a full time college student as well as a single mother who stays busy. I have previously used a HP All In one computer that I bought brand new a year ago and I hate that thing... It is so slow!!! When I first opened this item, I was just hoping that it would be a little faster! What I got instead was an amazing computer that is faster than I could have ever imagined. Now I don't use this thing for much more than amazon reviews, school work, and papers. But this is exactly what I needed.

A very reasonably priced laptop for basic computing needs. The specs that stick out to me for describing this as "basic needs" is 4GB of RAM, and 128GB M.2 SSD. Both are at the bare minimum in today's needs. Cell phones now come with those specs( hig

## Define the Reviews Dataset
Each item in the dataset will return a dictionary consisting of:


*   input_ids: the input token ids
*   attn_mask: the attention mask of the input sequence
*   label: the target star rating of the input review

In [120]:
class ReviewsDataset(Dataset):
    def __init__(self, df, max_length=1024):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length 
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # input=review, label=stars
        review = self.df.loc[idx, 'comment']
        # labels are 0-indexed
        label = int(self.df.loc[idx, 'stars']) - 1
        
        encoded = self.tokenizer(
            review,                      # review to encode
            add_special_tokens=True,
            max_length=self.max_length,  # Truncate all segments to max_length
            padding='max_length',        # pad all reviews with the [PAD] token to the max_length
            return_attention_mask=True,  # Construct attention masks.
            return_tensors='pt'
        )
        
        input_ids = encoded['input_ids']
        attn_mask = encoded['attention_mask']
        
        return {
            'input_ids': torch.tensor(input_ids),
            'attn_mask': torch.tensor(attn_mask), 
            'label': torch.tensor(label)
        }

Define some constants that are important later on.

In [121]:
MAX_LEN = 1024
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-05

In [122]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Create Datasets / DataLoaders
Create the train and test datasets and dataloaders for the neural network.

In [123]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

training_set = ReviewsDataset(train_dataset, MAX_LEN)
testing_set = ReviewsDataset(test_dataset, MAX_LEN)

print("# of samples in train set: {}".format(len(training_set)))
print("# of samples in test set: {}".format(len(testing_set)))

# of samples in train set: 96298
# of samples in test set: 24074


In [124]:
train_params = {
                'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {
                'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## Define the neural model for fine tuning
Given a review as an input sequence, we want to predict its star rating. This is a multi-class sequence classification task.

For out model, we will use BertForSequenceClassification and set the num_labels argument to the number of unique values for Amazon star ratings.

In [125]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(df['stars'].unique()), # number of unique labels for our multi-class classification problem
    output_attentions = False,
    output_hidden_states = False,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Fine Tuning the Model on Train Dataset

In [126]:
# Define the optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [127]:
# Define the accuracy function
def calcuate_accuracy(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [128]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _, data in enumerate(training_loader, 0):
        input_ids = data['input_ids'].to(device)
        mask = data['attn_mask'].to(device)
        labels = data['label'].to(device)

        print(input_ids)
        print(labels)
        print(mask)
        
        print('\n', input_ids.shape)
        print(labels.shape)
        print(mask.shape, '\n')
        
        loss, outputs = model(
                              input_ids, 
                              token_type_ids=None,
                              attention_mask=mask,
                              )
        break

        outputs = model(ids, mask)
        loss = loss_function(outputs, labels)
        tr_loss += loss.item()
        # gets labels with highest probabilities and their corresponding indices
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, labels)


        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # When using GPU
        optimizer.step()

    # print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    # epoch_loss = tr_loss/nb_tr_steps
    # epoch_accu = (n_correct*100)/nb_tr_examples
    # print(f"Training Loss Epoch: {epoch_loss}")
    # print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [129]:
model.to(device)
for epoch in range(EPOCHS):
    train(epoch)
    break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor([[[  101,  2572,  4039,  ...,     0,     0,     0]],

        [[  101,  2204,  2132,  ...,     0,     0,     0]],

        [[  101,  2573,  2092,  ...,     0,     0,     0]],

        ...,

        [[  101,  1045,  2074,  ...,     0,     0,     0]],

        [[  101,  2750,  1037,  ...,     0,     0,     0]],

        [[  101,  2009, 14057,  ...,     0,     0,     0]]], device='cuda:0')
tensor([2, 4, 4, 0, 4, 4, 4, 4, 0, 0, 0, 4, 4, 0, 3, 3, 4, 2, 4, 4, 0, 4, 4, 4,
        4, 4, 4, 0, 4, 4, 3, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        3, 3, 4, 1, 4, 4, 4, 4, 4, 4, 4, 0, 3, 0, 2, 2], device='cuda:0')
tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]], device='cuda:0')

 torch.Size([64, 1, 1024])
torch.Size([64])
torch.Size([64, 1, 1024]) 



RuntimeError: ignored